In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Installation of required libraries
import missingno as msno
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, GridSearchCV

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 

### Data Description
* Context
* "Predict behavior to retain customers. You can analyze all relevant customer data and develop focused customer retention programs."

* Content
* Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

* The data set includes information about:

* Customers who left within the last month – the column is called Churn
* Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
* Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
* Demographic info about customers – gender, age range, and if they have partners and dependents

* 1. Exploratory Data Analysis
* 2. Data Preprocessing & Feature Engineering
* 3. Modeling
* 4. Feature Selection
* 5. Hyperparameter Optimization with Selected Features
* 6. Submitting the results

* Customer ID
* Gender - Whether the customer is a male or a female
* SeniorCitizen - Whether the customer is a senior citizen or not (1, 0)
* Partner - Whether the customer has a partner or not (Yes, No)
* Dependents - Whether the customer has dependents or not (Yes, No)
* tenure - Number of months the customer has stayed with the company
* PhoneService - Whether the customer has a phone service or not (Yes, No)
* MultipleLines - Whether the customer has multiple lines or not (Yes, No, No phone service)
* InternetService - Customer’s internet service provider (DSL, Fiber optic, No)
* OnlineSecurity - Whether the customer has online security or not (Yes, No, No internet service)
* OnlineBackup - Whether the customer has online backup or not (Yes, No, No internet service)
* DeviceProtection - Whether the customer has device protection or not (Yes, No, No internet service)
* TechSupport - Whether the customer has tech support or not (Yes, No, No internet service)
* StreamingTV - Whether the customer has streaming TV or not (Yes, No, No internet service)
* StreamingMovies Whether the customer has streaming movies or not (Yes, No, No internet service)
* Contract - The contract term of the customer (Month-to-month, One year, Two year)
* PaperlessBilling - Whether the customer has paperless billing or not (Yes, No)
* PaymentMethod - The customer’s payment method (Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic))
* MonthlyCharges - The amount charged to the customer monthly
* TotalCharges - The total amount charged to the customer
* Churn - Whether the customer churned or not (Yes or No)


In [ ]:
#### Functions of the project.
def grab_col_names(dataframe, cat_th=10, car_th=20):
    """
    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.
    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri
    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi
    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))
    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı
    """


    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car

def cat_summary(dataframe, col_name,target, plot=False):
    print(f"#######{col_name.upper()}############")
    print(pd.DataFrame({col_name: dataframe[col_name].value_counts(),
                        "Ratio": 100 * dataframe[col_name].value_counts() / len(dataframe)}),end="\n\n\n")
    print(pd.DataFrame({"TARGET_MEAN": dataframe.groupby(col_name)[target].mean()}), end="\n\n\n")
    print("##########################################")
    if plot:
        sns.countplot(x=dataframe[col_name], data=dataframe)
        plt.show()
    print("##################END######################", end="\n\n\n")   


def num_summary(dataframe, numerical_col,target, plot=False):
    quantiles = [0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99]
    print(dataframe[numerical_col].describe(quantiles).T, end="\n\n\n")
    print(dataframe.groupby(target).agg({numerical_col: "mean"}), end="\n\n\n")  

    if plot:
        dataframe[numerical_col].hist(bins=20)
        plt.xlabel(numerical_col)
        plt.title(numerical_col)
        plt.show()
        


def target_summary_with_num(dataframe, target, numerical_col):
    print(dataframe.groupby(target).agg({numerical_col: "mean"}), end="\n\n\n")  
    
def rare_analyser(dataframe, target, cat_cols):
    for col in cat_cols:
        print(col, ":", len(dataframe[col].value_counts()))
        print(pd.DataFrame({"COUNT": dataframe[col].value_counts(),
                            "RATIO": dataframe[col].value_counts() / len(dataframe),
                            "TARGET_MEAN": dataframe.groupby(col)[target].mean()}).sort_values(by="COUNT"), end="\n\n\n")




def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df


def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

def missing_values_table(dataframe, na_name=False):
    """eksik değerlerin sayını ve oranını na_name= True olursa kolon isimleirinide verir.
    """
    na_columns = [col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    n_miss = dataframe[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[na_columns].isnull().sum() / dataframe.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio, 2)], axis=1, keys=['n_miss', 'ratio'])
    print(missing_df, end="\n")
    if na_name:
        return na_columns
    
    
def check_outlier(dataframe, col_name, q1=0.05, q3=0.95):
    """
    aykırı değer var mı yok sonucunu döner  
    """
    low_limit, up_limit = outlier_thresholds(dataframe, col_name, q1=q1, q3=q3)
    if dataframe[(dataframe[col_name] > up_limit) | (dataframe[col_name] < low_limit)].any(axis=None):
        return True
    else:
        return False    


def outlier_thresholds(dataframe, col_name, q1=0.25, q3=0.75):
    # üst v alt limitleri hesaplayıp döndürür.
    quartile1 = dataframe[col_name].quantile(q1)
    quartile3 = dataframe[col_name].quantile(q3)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable, q1=0.25, q3=0.75):
    """
    outliarları alt ve üst limite dönüştürüp baskılar.
    """
    low_limit, up_limit = outlier_thresholds(dataframe, variable, q1=q1, q3=q3)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit   
    


In [ ]:
######################################
# Exploratory Data Analysis
######################################

In [ ]:
# veri setinin okunması ve ilk gözlemlerin incelenmesi 
df_ = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df = df_.copy()
df.head()

In [ ]:
df.shape

In [ ]:
#veri seti hakkında genel bilgilerin incelenmesi 
print("##################### Shape #####################")
print(df.shape)
print("##################### Types #####################")
print(df.dtypes)
print("##################### Head #####################")
print(df.head())
print("##################### Tail #####################")
print(df.tail())
print("##################### NA #####################")
print(df.isnull().sum())
print("##################### INFO #####################")
print(df.info())
print("##################### NumUnique #####################")
print(df.nunique())
print("##################### Describe #####################")
print(df.describe())

# Açıklamalar;
* Veri hiç eksik gözlem olmadığı görülüyor ama yaptığım ilerideki işlemler ile TotalChaerges değişkeni 10 tane eksik gözleme sahip. Burada eksikliği rassalıktan kaynaklanıp kaynaklanmadığı araştırmalıyız. 
* Tenure değeri 0 olanlar dikkat çekiyor incelenmesi gerekir. 
* Değişkenlerin büyük bir bölümü kategorik değişkenlerden oluşuyor. 
* SeniorCitizen değikenini numerik tipte kategorik değere dönüştüreceğim.
* TotalCharges değişkeni kategorik değişken olarak görünüyor numerik değişkene çevireceğim
* Analiz yapabilmek için hedef değişken olan Chrun değerini numerik değişkene çevireceğim

In [ ]:
# yazdığım fonksiyon ile değişkenleri tiplerine göre listelere ayırıyorum
cat_cols, num_cols, cat_but_car = grab_col_names(df)

In [ ]:
# 'TotalCharges' veri tipini float yapacağım.
cat_but_car

In [ ]:
# Kategorik değişkenler
cat_cols

In [ ]:
# numerical columns
num_cols

In [ ]:
#"TotalCharges" değişkenini numerik formata dönüştürdüm
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"],errors = 'coerce')
num_cols.append("TotalCharges") 

In [ ]:
#Churn analizine bir etkisi olmadığı için "customerID" sütununu veri setinden çıkarıyorum.
df.drop("customerID",axis=1, inplace=True)

In [ ]:
# Data setimizi biraz dengesiz olarak görünüyor burada veri seti dengelenebilir veya daha fazla veri ile analiz
# yapılabilir. şimdilik bu durumu göz ardı ediyorum.
df["Churn"].value_counts()

In [ ]:
# yukarda da tenure değeri sıfır olanların araştırmamız gerektiğinden bahsetmiştim
# burda anlaşılacağı üzere TotalCharges değişkenindeki na değerler rassal değil veri setine yeni gelen müşteriler 
# olduğu anlaşılıyor. burda bu na değerlerini ilgili gözlem biriminin MonthlyCharges değeri ile dolduracağım çünkü henüz ödemeye
#başladıkları için bu değer ile doldurmak mantıklı.
df[df["TotalCharges"].isnull()]

In [ ]:
#dediğim gibi indexlerde tutuyor.
df[df["TotalCharges"].isnull()].index

In [ ]:
df[df["tenure"] == 0].index

In [ ]:
# yukarda değidiğim gibi tenure lar 0 olduğu için ilk aylık ödemeleri ile totalcharges değişkenini doldurdum.
df.loc[df["tenure"] == 0, "TotalCharges"] = df.loc[df["tenure"] == 0, "MonthlyCharges"]

In [ ]:
# na olan gözlem birimleri dolduruldu.
df.loc[df["tenure"] == 0, "TotalCharges"]

In [ ]:
# SeniorCitizen değişkeninin tipini object yapıyorum.
df['SeniorCitizen']= df['SeniorCitizen'].astype(object)

In [ ]:
#sayısal değişkenlerin aralarındaki korelasyonun incelenmesi
plt.figure(figsize=(25,20))
mask = np.zeros_like(df.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(df.corr(),cmap="jet",annot=True,linewidths=0, linecolor='white',cbar=True,mask=mask)

In [ ]:
# I convert to Churn values to binary values so if a customer churn it will represented by 1 otherwise 0
# I also convert Churn data type to integer to analyze categorical columns with Churn

df["Churn"] = df["Churn"].apply(lambda x: 1 if x=='Yes' else 0) 
df['Churn'].astype(int)
for col in cat_cols:
    
    cat_summary(df, col,"Churn")

In [ ]:
df.groupby(['gender',
            'Partner',
            'SeniorCitizen',
            'Dependents',
            'PaperlessBilling',
            'PhoneService',
            'MultipleLines',
            'InternetService',
            'OnlineSecurity',
            'OnlineBackup',
            'DeviceProtection',
            'TechSupport',
            'StreamingTV',
            'StreamingMovies',
            'Contract',
            'PaymentMethod']).agg({'Churn': 'mean', 'tenure': 'mean'})

In [ ]:
# summurazing numerical variables(numerik değerlerin analiz edilmesi)
for col in num_cols:
    target_summary_with_num(df, "Churn", col)

### Açıklamalar ve Firmaya Öneriler;
* Gender -  değişkenindeki farklılığın Churn olup olamamaya herhangi bir etkisi bulunmamaktadır.

* SeniorCitizen - yaşlı müşterilerin daha fazla churn oldukları gözlemlenmektedir. Bunun nedeni araştırılmalıdır. Müşteriler artık telekom hizmetlerini kullanamadıkları için yani firma dışı nedenlerden mi churnm oluyolar.

* Partner - Partnerı olan müşteriler daha az churn olma eğilimindeler burada firma partnerlara yönelik kampanyalar yaparak partner şeklinde müşteri kazanıp müşterilerin churn olmasını azaltabilir.

* Dependents - bakmakla yükümlü oldukları insanlar olan kişiler olmayanlara %50 daha az churn oluyorlar.

* Tenure - Müşteri tenure(şirket yaşı) artıkça churn olma oranı düşüyor. churn olanlar genelde tenure düşük olan müşteriler churn olma eğiliminde firma başlangıçta bu müşterilerin churn olmasını engellemek için belirli aralıklarla hatırlatma veya avantajlı bir kampanyalar ile daha uzun sözleşmeler yapmalıdır.

* Phoneservice - değişkenindeki farklılığın Churn olup olamamaya herhangi bir etkisi bulunmamaktadır.

* MultipleLines - değişkenindeki farklılığın Churn olup olamamaya belirgin bir etkisi bulunmamaktadır.

* InternetService - burda fiber obtik servisi olan müşterilerin churn olma eğiliminin yüksek olduğu görülüyor. Bu müşterilerin teknolojiyi ve trendleri yakından taqkip ettiklerini ve daha fazla duyarlı olduklarını düşünüyorum bu müşterilere hem yeni teknolojiler ile ilgili bilgi akışı sağlanmalı ve bu müşterileri elde tutmak için farklı yan hizmetler verilebilir mesela bir teknoloji dergisi aboneliği veya trend olan bir site veya uygulamadan ücretsiz yararlanma gibi farklı alternatifler ile firma bağlılığı kazandırılmalıdır. Bu durum aynı zamanda Telco firmasının teknolojiyi çok rekabetin olduğu bu piyasada iyi bir şekilde takip etmesi gerektiğine de işaret ediyor. 

* Onlinesecurity - online güvenliği olmayan müşterilerin yüksek churn olma eğiliminde oldukları görülmüştür. Bu hizmetin günümüz internet ortamının güvensizliği de tanıtılarak ve gerekirse bu hizmetin uygun sağlanması için çeşitli kampanyalar düzenlerek müşterilerek bu ürünün satılması müşteri kaybını önlemede büyük destek sağlayacaktır.

* OnlineBackup - Online backup da online güvenlik ile aynı özelliği gösteriyor ve benzer önlem ve staretejiler bu değişken içinde yapılabilir.

* DeviceProtection - bu değişken Onlinesecurity ve OnlineBackup ile benzer şekilde etkiye sahip. Bu üç ürünü veya en az ikisini çapraz satış yaparak müşterilerin churn olmasına engel olmada büyük başarı elde edilebilir.

* TechSupport - teknolojik desteği olmayan müşterilerin churn olma oranları yüksek, burda firma müşterilere bir şekilde ulaşarak hem yerinde hemde online teknolojik destek olmalıdır.

* StreamingTV - internet servisi olmayan müşterilere hariç değişkenindeki farklılığın Churn olup olamamaya herhangi bir etkisi bulunmamaktadır.

* StreamingMovies - internet servisi olmayan müşterilere hariç değişkenindeki farklılığın Churn olup olamamaya herhangi bir etkisi bulunmamaktadır.

* Contract - kontrat süreleri artıkça churn oranları düşmektedir. Firma uzun kontratlarda rakabetçi ve kısa süreli kontratlara göre uygun tarifeler vererek müşteri bağlılığını artırmalıdır. 

* PaperlessBilling - kağıt fatura alan müşterilerin churn oranı daha düşük görünmekte olup firma kağıt ve posta maliyetinin yüksek olduğu düşünülürse kağıt fatura almayan müşterilerin bilgilendirmelerini daha etkili bir şekilde yapabilir. 

* PaymentMethod - electronik check ödemesi olan müşterilerin churn oranı yüksek ve özellikle kredi kartı otomatik ödeme talimatları churn oranları düşüktür müşterileri bu kanala itmek müşterinin fiyat duyarlılığını bir nebze azaltacaktır hesaba verilen otomatik ödemeden ziyade çünkü hesap hareketleri kredi kartı ekstresinden daha çok kontrol edilir. 

* MonthlyCharges - aylık sabit ödemeleri yüksek olan müşterilerin churn oranları yüksek burda aylık sabit ödemenin düşük tutulup müşteriye çapraz satış yapılarak gelir artışı sağlamak churn olmayı azaltabilir çünkü totalcharges değişkenin yüksek olması churn olmaya etkisi az olanlara göre düşük.


In [ ]:
################################################
# Feature Engineering
################################################

In [ ]:
# Müşterinin firmaya düzenli ödeme yaptığına dair bir değişken 
df['Engaged']=1 
df.loc[(df['Contract']=='Month-to-month'),'Engaged']=0
df['Engaged'] = df['Engaged'].astype(object)

# ödeme yöntemi elektronik ve müşterinin bize bağlılığı olmadığını gösterir bir değişken oluşturdum(churn yüksekti)
df['ElectCheck']=0 
df.loc[(df['PaymentMethod']=='Electronic check') & (df['Engaged']==0),'ElectCheck']=1
df['ElectCheck'] = df['ElectCheck'].astype(object)

# internet bağlantısı fiber optik olanlar churn olmayı ço;Ük etkilediği için burdanda yeni bir değişken oluşturudm
df['fiberopt']=1 
df.loc[(df['InternetService']!='Fiber optic'),'fiberopt']=0
df['fiberopt'] = df['fiberopt'].astype(object)

# hiç koruması olmayan müşteriler.
df['NoProt']=1 
df.loc[(df['OnlineBackup']!='No') | (df['DeviceProtection']!='No') | (df['TechSupport']!='No'),'NoProt']=0
df['NoProt'] = df['NoProt'].astype(object)

# koruma düzeyi ekledim eğer online security , onlinebackup ve device protection varsa yüksek koruma dedim.
df["Protection_level"] = np.where((df["OnlineSecurity"] == "Yes") & (df["OnlineBackup"] == "Yes") & (df["DeviceProtection"] == "Yes"), "High","low")

#Ürün sayısı değişkeni ekliyorum
df2 = df_.copy() #data framemimdeki değişkenler etkilenmesin diye df2 isimli datasette yaptım işlemleri
df["Numberofproduct"] = 0
for col in ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies']:
    df2[col] = df2[col].apply(lambda x: 1 if x=='Yes' else 0) 
    df2[col].astype(int)
    df["Numberofproduct"] += df2[col]
    
# Tenure değişkeni feature importanceda en üstte çıktığı için ağırlıklandırmak için ekstra yeni değişken oluşturdum.
df['tenure_seg'] = pd.qcut(df['tenure'], 3, labels=["C", "B","A"]) 
df['tenure_seg'] = df['tenure_seg'].astype(object)


In [ ]:
# yazdığım fonksiyon ile değişkenleri tiplerine göre listelere ayırıyorum
cat_cols, num_cols, cat_but_car = grab_col_names(df)

In [ ]:
cat_cols

In [ ]:
#df.groupby(['tenure_seg']).agg({"tenure":"mean"})

In [ ]:
# yeni oluşturduğum bu değişkenin etksinin ordinal bir şekilde yanıltılmasını istediğim için
# farklı gözlem birimi az olmasına rağmen numerik olarak bıraktım
num_cols.append("Numberofproduct")
cat_cols = [col for col in cat_cols if col not in ["Numberofproduct","Churn", "tenure_seg"]]

In [ ]:
cat_cols

In [ ]:
#yeni feature oluşturduktan sonra datayı genel olarak gözlemliyorum.
print("##################### Shape #####################")
print(df.shape)
print("##################### Types #####################")
print(df.dtypes)
print("##################### Head #####################")
print(df.head())
print("##################### Tail #####################")
print(df.tail())
print("##################### NA #####################")
print(df.isnull().sum())
print("##################### INFO #####################")
print(df.info())
print("##################### NumUnique #####################")
print(df.nunique())
print("##################### Describe #####################")
print(df.describe())

In [ ]:
# beklenildiği gibi yeni oluşturduğum featureların kategorik değişken 
#üzerinden beklenildiği etkiyi gözlemleyebiliyoruz.
for col in cat_cols:
    
    cat_summary(df, col,"Churn", plot=True)

In [ ]:
# tenure segmentin ordinal olduğu varsayıyorum ve o yüzden labelencode ediyorum.
# 0 değeri A segmentine karşılık gelicek
labelencoder = LabelEncoder()
df['tenure_seg'] = labelencoder.fit_transform(df['tenure_seg'])

In [ ]:
# algoritmaların daha hızlı çalışması için numerik değişkenleri scale ediyorum.
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [ ]:
# one_hot_encoder ve dummy değişkenlerin oluşturulması 
df = one_hot_encoder(df, cat_cols, drop_first=True)

In [ ]:
# değişken sayımız 38 e çıktı
df.head()

In [ ]:
def base_models(X, y, scoring="roc_auc"):
    print("Base Models....")
    classifiers = [('LR', LogisticRegression()),
                   ('KNN', KNeighborsClassifier()),
                   ("SVC", SVC()),
                   ("CART", DecisionTreeClassifier()),
                   ("RF", RandomForestClassifier()),
                   ('Adaboost', AdaBoostClassifier()),
                   ('GBM', GradientBoostingClassifier()),
                   #('XGBoost', XGBClassifier()),
                   ('LightGBM', LGBMClassifier()),
                   # ('CatBoost', CatBoostClassifier(verbose=False))
                   ]

    for name, classifier in classifiers:
        cv_results = cross_validate(classifier, X, y, cv=3, scoring=scoring)
        print(f"{scoring}: {round(cv_results['test_score'].mean(), 4)} ({name}) ")

In [ ]:
X = df.drop(['Churn'], axis=1)
y = df['Churn']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.05)

In [ ]:
base_models(X_train, y_train, scoring="roc_auc")

In [ ]:
base_models(X_test, y_test, scoring="roc_auc")

In [ ]:
######################################################
# Automated Hyperparameter Optimization
######################################################
lgr_params = {"solver" : ['newton-cg', 'liblinear'],
              "penalty" : ['l2'],
              "C" : [100,1, 0.01]}
gbm_params = {'learning_rate': [0.1, 0.01,1],
             'max_depth': [1,2,4],
             'n_estimators': [100,1000,2000],
             'subsample': [.5,.75,1]}

knn_params = {"n_neighbors": range(2, 50)}

cart_params = {'max_depth': range(1, 20),
               "min_samples_split": range(2, 30)}

rf_params = {"max_depth": [8, 15, None],
             "max_features": [5, 7, "auto"],
             "min_samples_split": [15, 20],
             "n_estimators": [200, 300]}



lightgbm_params = {"learning_rate": [0.01, 0.1, 0.001],
                   "n_estimators": [300, 500, 1500],
                   "colsample_bytree": [0.7, 1]}
xgboost_params = {
                "max_depth": [3, 5, 8, 12, 15],
                "learning_rate": [0.5,0.15,0.30],
                "min_child_weight": [1, 3, 5, 7],
                "gamma": [0.0, 0.2, 0.4],
                "colsample_bytree": [0.3, 0.4, 0.7]}

classifiers = [('KNN', KNeighborsClassifier(), knn_params),
               ("CART", DecisionTreeClassifier(), cart_params),
               ("RF", RandomForestClassifier(), rf_params),
               #('XGBoost', XGBClassifier(), xgboost_params),
               ('LightGBM', LGBMClassifier(), lightgbm_params),
              ('GBM', GradientBoostingClassifier(),gbm_params),
              ('LR', LogisticRegression(), lgr_params)]

In [ ]:
def hyperparameter_optimization(X, y, cv=3, scoring="roc_auc"):
    """verilen parametre giridlerine göre cross validasyon yapar ve en iyi parametleri tutan 
    modellerin olduğu listeyi return eder.
    """
    print("Hyperparameter Optimization....")
    best_models = {}
    for name, classifier, params in classifiers:
        print(f"########## {name} ##########",end="\n\n")
        print("########## before ##########",end="\n\n")
        cv_results = cross_validate(classifier, X, y, cv=3, scoring=["accuracy", "f1", "roc_auc"])
        print(f"Accuracy: {cv_results['test_accuracy'].mean()}")
        print(f"F1Score: {cv_results['test_f1'].mean()}")
        print(f"ROC_AUC: {cv_results['test_roc_auc'].mean()}", end="\n\n")
        
        print("########## after ##########",end="\n\n")
        gs_best = GridSearchCV(classifier, params, cv=cv, n_jobs=-1, verbose=False).fit(X, y)
        final_model = classifier.set_params(**gs_best.best_params_)

        cv_results = cross_validate(final_model, X, y, cv=cv, scoring=scoring)
        print(f"Accuracy: {cv_results['test_accuracy'].mean()}")
        print(f"F1Score: {cv_results['test_f1'].mean()}")
        print(f"ROC_AUC: {cv_results['test_roc_auc'].mean()}")
        print(f"{name} best params: {gs_best.best_params_}", end="\n\n")
        best_models[name] = final_model
    return best_models


In [ ]:
# hiper parametre optimizasyonu ve optimize edilmiş train hataları
best_models = hyperparameter_optimization(X_train, y_train, cv=3, scoring=["accuracy", "f1", "roc_auc"])

In [ ]:
# test hatalarının hesaplanması için yazdığım fonksiyon
def test_scores(best_models, X_test, y_test):
    """void fonksiyon modellerin olduğu model listesi girildiğinde her model için fit etme
    işlemi yapar ve sonrasında test setinin tahminlerini yaparak model başarı değerlendirme
    metrikleri ile confusion matrisi oluşturur. 
    """
    for i, j in best_models.items():
        print(i)
        j.fit(X_train, y_train)
        predictions=j.predict(X_test)
        print(confusion_matrix(y_test,predictions), end="\n\n")
        print(classification_report(y_test, predictions),end="\n\n")
        print(accuracy_score(y_test, predictions),end="\n\n")
        

In [ ]:
# tune edilmiş parametleri taşıyan modellerin olduğu liste
best_models

In [ ]:
# confusion matrisin oluşturulması ve detaylı raporlanması
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, roc_auc_score

test_scores(best_models, X_test, y_test)

In [ ]:
######################################################
# Stacking & Ensemble Learning
######################################################

# en iyi değerleri veren modeller ile birleştirilmiş modelin kurulması

def voting_classifier(best_models, X, y):
    """
    En iyi değeri veren modelleri alıp birleştirilmiş modeli döner ve model değerlendirme metriklerini
    print eder.
    """
    print("Voting Classifier...")
    voting_clf = VotingClassifier(estimators=[('KNN', best_models["KNN"]), ('LR', best_models["LR"]),
                                              ('RF', best_models["RF"])],
                                  voting='soft').fit(X, y)
    cv_results = cross_validate(voting_clf, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])
    print(f"Accuracy: {cv_results['test_accuracy'].mean()}")
    print(f"F1Score: {cv_results['test_f1'].mean()}")
    print(f"ROC_AUC: {cv_results['test_roc_auc'].mean()}")
    return voting_clf

In [ ]:
voting_clf = voting_classifier(best_models, X, y)

In [ ]:
voting_clf

In [ ]:
voting_clf.predict(X_test)

In [ ]:
# ensemble modelin değerlendirilmesi
voting_clf.fit(X_train, y_train)
predictions=voting_clf.predict(X_test)
print(confusion_matrix(y_test,predictions), end="\n\n")
print(classification_report(y_test, predictions),end="\n\n")
print(accuracy_score(y_test, predictions),end="\n\n")

In [ ]:
#######################################
# Feature Selection
#######################################

In [ ]:
def plot_importance(model, features, num=len(X), save=False):
    """
    feature önemlerini gösterir.
    """
    feature_imp = pd.DataFrame({'Value': model.feature_importances_, 'Feature': features.columns})
    plt.figure(figsize=(30, 30))
    sns.set(font_scale=1)
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value",
                                                                     ascending=False)[0:num])
    plt.title('Features')
    plt.tight_layout()
    plt.show()
    if save:
        plt.savefig('importances.png')
    return   

In [ ]:
model = LGBMClassifier(colsample_bytree=0.7, learning_rate=0.01, n_estimators=300).fit(X_train, y_train)
feature_imp = pd.DataFrame({"Feature": X.columns, "Value": model.feature_importances_ >0}).sort_values(by="Value", 
                                                                                                 ascending=False)

In [ ]:
# model önem düzeyleri 
plot_importance(model, X)

* Burda yeni ürettiğim değişkenlerin bir çoğu ile yaramış....

In [ ]:
# önem düzeyi sıfırdan büyük featureların belirlenmesi ve bunlar ile model kurulması
feature_imp[feature_imp["Value"] > 0].shape

feature_imp[feature_imp["Value"] < 1].shape # birden küçük olanlar


zero_imp_cols = feature_imp[feature_imp["Value"] < 1]["Feature"].values


selected_cols = [col for col in X.columns if col not in zero_imp_cols]

In [ ]:
#######################################
# Hyperparameter Optimization with Selected Features
#######################################
voting_clf = voting_classifier(best_models, X[selected_cols], y)


In [ ]:
# model tahminleri
voting_clf.predict(X_test[selected_cols])

In [ ]:
# ensemble modelin değerlendirilmesi
voting_clf.fit(X_train, y_train)
predictions=voting_clf.predict(X_test)
print(confusion_matrix(y_test,predictions), end="\n\n")
print(classification_report(y_test, predictions),end="\n\n")
print(accuracy_score(y_test, predictions),end="\n\n")

In [ ]:
#Aritifial Neural Networks:
# bu veri setinde ANN kurulması ve değerlendirilmesi.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
ann = Sequential()
ann.add(Dense(units = 200,activation="relu", kernel_regularizer=l2(0.001)))
ann.add(Dropout(0.2))
ann.add(Dense(units = 100, activation="relu",kernel_regularizer=l2(0.001)))
ann.add(Dropout(0.2))
ann.add(Dense(units = 50, activation="relu",kernel_regularizer=l2(0.001)))
ann.add(Dense(1,activation="sigmoid")) 
ann.compile(optimizer = "adam", loss="binary_crossentropy",metrics=["accuracy"])
callback=EarlyStopping(monitor="val_loss", patience=2)
history = ann.fit(x = X_train[selected_cols], y= y_train, validation_data=(X_test[selected_cols],y_test), batch_size=16, epochs=100,callbacks=[callback])

In [ ]:
# model test tain değerlerinin izlenmesi 
sns.set_style("darkgrid")
pd.DataFrame(ann.history.history).plot(figsize=(15,10))

In [ ]:
predictions = ann.predict_classes(X_test[selected_cols])
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test, predictions))
print(accuracy_score(y_test, predictions))